In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

<ipython-input-1-b05d1717aef3>:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [319]:
df = pd.read_csv('bank-full.csv',sep=';')

df.columns =df.columns.str.lower().str.replace(' ','_')
print("--------------------------- This is dataframe----------------------------------")
print("\n",df.head().T)
#print("-----------------------------this is data types-------------------------------")
#print("this is data types\n",df.dtypes)
print("-------------------------------------------------------------")

#replacing the boolean yes no with numeric value in the Y variable
df['y'] = (df['y'] =='yes').astype(int)
#print(df['y'].tail(25))

#df['poutcome'].unique()

--------------------------- This is dataframe----------------------------------

                     0           1             2            3        4
age                58          44            33           47       33
job        management  technician  entrepreneur  blue-collar  unknown
marital       married      single       married      married   single
education    tertiary   secondary     secondary      unknown  unknown
default            no          no            no           no       no
balance          2143          29             2         1506        1
housing           yes         yes           yes          yes       no
loan               no          no           yes           no       no
contact       unknown     unknown       unknown      unknown  unknown
day                 5           5             5            5        5
month             may         may           may          may      may
duration          261         151            76           92      198
campaign

# Question 1
What is the most frequent observation (mode) for the column education?

In [346]:
edu=df['education'].mode()
print("This is 'mode' for column 'Education'\n", edu)

This is 'mode' for column 'Education'
 0    secondary
Name: education, dtype: object


# Question 2
What are the two features that have the biggest correlation?

In [349]:
from sklearn.model_selection import train_test_split
df3=df.copy()
#print(df3)
#df3=df3.drop(['y'], axis=1)
df3=df3
df_train_full, df_test = train_test_split(df3, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)
y_train = df_train.y.values
y_val = df_val.y.values
y_test =df_test.y.values
del df_train['y']
del df_val['y']
df_train_full.isnull().sum()
df_train_full.y.value_counts()
global_mean = df_train_full.y.mean()
print('this is global mean',round(global_mean, 3))
numerical = ['age','balance','day','campaign','pdays','previous']
print("correlation of pdays with previous -->",df_train_full['pdays'].corr(df_train_full.previous))
print("correlation of day with campaign -->",df_train_full['day'].corr(df_train_full.campaign))
print("correlation of age with balance -->",df_train_full['age'].corr(df_train_full.balance))
print("correlation of day with pdays -->",df_train_full['day'].corr(df_train_full.pdays))


this is global mean 0.117
correlation of pdays with previous --> 0.43721982236032464
correlation of day with campaign --> 0.16565234398276027
correlation of age with balance --> 0.10259640144442296
correlation of day with pdays --> -0.0918694162196096


# Question 3

## Which of these variables has the biggest mutual information score?

In [350]:
from sklearn.metrics import mutual_info_score
categorical = ['job','marital','education','default','housing','loan','contact','month','poutcome']
df_train_full[categorical].nunique()

df_group1 = df_train_full.groupby(by='education').y.agg(['mean'])
print('\n',round(df_group1,2))
df_group2 = df_train_full.groupby(by='poutcome').y.agg(['mean'])
print('\n',round(df_group2,2))
df_group3 = df_train_full.groupby(by='contact').y.agg(['mean'])
print('\n',round(df_group3,2))
df_group4 = df_train_full.groupby(by='housing').y.agg(['mean'])
print('\n',round(df_group4,2))

def calculate_mi(series):
    return mutual_info_score(series, df_train_full.y)

df_mi = df_train_full[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='Mutual Information')


#display(df_mi.head())
#display(df_mi.tail())
display(df_mi)




            mean
education      
primary    0.09
secondary  0.11
tertiary   0.15
unknown    0.14

           mean
poutcome      
failure   0.13
other     0.16
success   0.65
unknown   0.09

            mean
contact        
cellular   0.15
telephone  0.14
unknown    0.04

          mean
housing      
no       0.17
yes      0.08


,Mutual Information
poutcome,0.028893
month,0.024718
contact,0.013041
housing,0.009551
job,0.008437
loan,0.002641
education,0.002564
marital,0.001980
default,0.000407


# Question 4
## What accuracy did you get?

In [351]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
#print(df_train)
train_dict = df_train[categorical + numerical].to_dict(orient='records')
train_dict[0]
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)
X_train = dv.transform(train_dict)
X_train.shape
dv.get_feature_names_out()
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
#model = LogisticRegression(solver='liblinear', random_state=1)
model.fit(X_train, y_train)
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)
model.predict_proba(X_val)

y_pred = model.predict_proba(X_val)[:, 1]
y_pred
churn = y_pred > 0.5
acc=(y_val == churn).mean()
round(acc,4)

0.8949

# Question 5
## Which of following feature has the smallest difference?

In [352]:
least=df_train_full[numerical].corrwith(df_train_full.y).abs().sort_values(ascending=False)
print('list of features\n',least)
print(f"least important feature is age : {least.iloc[5]}")
#least.iloc[5]

list of features
 pdays       0.101045
previous    0.087392
campaign    0.072134
balance     0.055252
day         0.030624
age         0.024931
dtype: float64
least important feature is age : 0.02493100896982028


# Question 6
## Which of these C leads to the best accuracy on the validation set?

In [353]:
full_train=df_train_full[categorical + numerical].to_dict(orient='records')
dv=DictVectorizer(sparse=False)
X_full_train=dv.fit_transform(full_train)
Y_full_train=df_train_full.y.values
model = LogisticRegression(solver='liblinear', C=100, max_iter=1000, random_state=42)
model.fit(X_full_train,Y_full_train)
dicts_test=df_test[categorical + numerical].to_dict(orient='records')
X_test=dv.transform(dicts_test)
y_pred= model.predict_proba(X_test)[:,1]
fd_decision = (y_pred > 0.5)
(fd_decision == y_test).mean()

0.893177043016698